## Deliverable 2. Create a Customer Travel Destinations Map.

In [13]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import the API key.
import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [14]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,College,US,64.8569,-147.8028,56.80,74,75,3.44,broken clouds
1,1,San Andres,CO,12.5847,-81.7006,86.02,74,40,12.66,scattered clouds
2,2,Iracoubo,GF,5.4802,-53.2011,82.85,72,39,8.57,scattered clouds
3,3,Mount Gambier,AU,-37.8333,140.7667,47.23,85,62,8.05,broken clouds
4,4,Harper,LR,4.3750,-7.7169,77.50,66,66,8.68,broken clouds


In [15]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Iracoubo,GF,5.4802,-53.2011,82.85,72,39,8.57,scattered clouds
4,4,Harper,LR,4.3750,-7.7169,77.50,66,66,8.68,broken clouds
8,8,Puerto Narino,CO,-3.7703,-70.3831,83.66,70,66,3.31,light rain
11,11,Caravelas,BR,-17.7125,-39.2481,77.49,63,2,8.46,clear sky
12,12,Rikitea,PF,-23.1203,-134.9692,74.55,73,36,12.15,scattered clouds
19,19,Anlu,CN,31.1661,112.5831,82.31,84,99,9.24,overcast clouds
21,21,Nchelenge,ZM,-9.3451,28.7340,73.58,50,50,6.20,scattered clouds
26,26,Guerrero Negro,MX,27.9769,-114.0611,75.16,62,61,14.72,broken clouds
27,27,Puerto Ayora,EC,-0.7393,-90.3518,75.16,88,82,7.00,broken clouds
33,33,Lorengau,PG,-2.0226,147.2712,80.28,78,98,4.99,overcast clouds


In [17]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_no_nulls_df = preferred_cities_df.dropna() 

In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_no_nulls_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Iracoubo,GF,82.85,scattered clouds,5.4802,-53.2011,
4,Harper,LR,77.50,broken clouds,4.3750,-7.7169,
8,Puerto Narino,CO,83.66,light rain,-3.7703,-70.3831,
11,Caravelas,BR,77.49,clear sky,-17.7125,-39.2481,
12,Rikitea,PF,74.55,scattered clouds,-23.1203,-134.9692,
19,Anlu,CN,82.31,overcast clouds,31.1661,112.5831,
21,Nchelenge,ZM,73.58,scattered clouds,-9.3451,28.7340,
26,Guerrero Negro,MX,75.16,broken clouds,27.9769,-114.0611,
27,Puerto Ayora,EC,75.16,broken clouds,-0.7393,-90.3518,
33,Lorengau,PG,80.28,overcast clouds,-2.0226,147.2712,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    try:
    
        # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping ")
        
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
-----------------------------
Data Retrieval Complete      
-----------------------------


In [21]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Iracoubo,GF,82.85,scattered clouds,5.4802,-53.2011,Le Gîte d’Iracoubo
4,Harper,LR,77.50,broken clouds,4.3750,-7.7169,Screensaver
8,Puerto Narino,CO,83.66,light rain,-3.7703,-70.3831,Eware Refugio Amazonico
11,Caravelas,BR,77.49,clear sky,-17.7125,-39.2481,Pousada dos Navegantes
12,Rikitea,PF,74.55,scattered clouds,-23.1203,-134.9692,People ThankYou
...,...,...,...,...,...,...,...
708,San Quintin,MX,73.81,overcast clouds,30.4833,-115.9500,Don Eddie's Sport Fishing Center
709,Toyooka,JP,74.57,scattered clouds,35.5333,134.8333,O Hotel Toyooka
710,Penha,BR,70.48,scattered clouds,-26.7694,-48.6458,Hotel Mirante do Parque
715,Tubmanburg,LR,79.56,overcast clouds,6.8708,-10.8172,LITTLE WAY GUEST HOUSE


In [22]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(50.0, -20.0), zoom_level=3)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# # 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))